In [357]:
import numpy as np
import matplotlib.pyplot as plt

In [358]:
REPRESENTATION_LENGTH = 11 

In [359]:
def number_to_representation(number, length):
    representation = []
    for i in range(length - 1, -1, -1):
        if number & (1 << i):
            representation.append(1)
        else:
            representation.append(0) 
    return representation

In [360]:
class Hypothesis: 
    def __init__(self, length, representation = None):
        if representation == None:
            self.__hypothesis = number_to_representation(np.random.randint(0, (1 << length)), length)
            self.__length = length
        else:
            self.__hypothesis = representation
            self.__length = length
    
    
    def get_representation(self):
        return self.__hypothesis

    def classify(self, instance):
        result = self.__hypothesis[self.__length - 1]

        for i in range(len(instance)):
            if instance[i] == 1 and self.__hypothesis[i] == 0:
                return result ^ 1
        
        return result

In [361]:
class Fitness:

    def __init__(self, instances, targets):
        self.__instances = instances
        self.__targets = targets
    
    def __call__(self, hypothesis):
        result = 0

        for instance, target in zip(self.__instances, self.__targets):
            if hypothesis.classify(instance) == target:
                result += 1
        
        return result / len(self.__instances)


In [362]:
class GA:
    def __init__(self, length, fitness, p = 30, r = 0.6, m = 0.001, threshold = 0.25):
        self.__p = p
        self.__r = r
        self.__m = m
        self.__length = length
        self.__fitness = fitness
        self.__threshold = threshold

        self.__init_hypothesis()
    
    def __init_hypothesis(self):
        self.__hypothesis = []

        for _ in range(self.__p):
            self.__hypothesis.append(Hypothesis(length = self.__length))
    
    @classmethod
    def __crossover(cls, hypothesis1, hypothesis2):
        representation1 = hypothesis1.get_representation()
        representation2 = hypothesis2.get_representation()
        length = len(representation1)

        n1 = np.random.randint(0, length - 2)
        n2 = np.random.randint(n1 + 2, length)

        left1 = representation1[:(n1 + 1)]
        left2 = representation2[:(n1 + 1)]

        for i in range(n1 + 1):
            representation1[i] = left2[i]
            representation2[i] = left1[i]
        
        right1 = representation1[n2:]
        right2 = representation2[n2:]

        for i in range(n2, len(representation1)):
            representation1[i] = right2[i - n2]
            representation2[i] = right1[i - n2]
        
        return [Hypothesis(representation = representation1, length = len(representation1)), Hypothesis(representation = representation2, length = len(representation2))]
    
    @classmethod
    def __mutate(cls, hypothesis):
        representation = hypothesis.get_representation()

        position = np.random.randint(0, len(representation))
        representation[position] ^= 1

        return Hypothesis(representation = representation, length = len(representation))
    
    def __probabilities(self):
        probabilities = []

        for hypothesis in self.__hypothesis:
            probabilities.append(self.__fitness(hypothesis))
        
        sum_fitness = sum(probabilities)
        probabilities = [p / sum_fitness for p in probabilities]
        
        return probabilities

    def __sample_hypothesis(self, probabilities, number):
        hypothesis = list(np.random.choice(self.__hypothesis, number, p = probabilities, replace = False))

        return hypothesis

    def __select_from_current(self, probabilities):
        return self.__sample_hypothesis(probabilities, int((1 - self.__r) * self.__p))

    def __select_pairs(self, probabilities):
        number = int(self.__r * self.__p)
        number += number % 2

        hypothesis = np.random.choice(self.__hypothesis, int(self.__r * self.__p), p = probabilities)
        crossovers = []

        for i in range(0, number, 2):
            crossovers.extend(GA.__crossover(hypothesis1 = hypothesis[i], hypothesis2 = hypothesis[i + 1]))
        
        return crossovers

    def __mutate_hypothesis(self, hypothesis):
        values = np.random.binomial(n = 1, p = self.__m, size = len(hypothesis))
        assert len(values) == len(hypothesis)

        for i in range(len(hypothesis)):
            if values[i] == 1:
                hypothesis[i] = GA.__mutate(hypothesis[i])
        
        return hypothesis
    
    def __metric(self):
        maximum = 0
        for hypothesis in self.__hypothesis:
            maximum = max(maximum, self.__fitness(hypothesis))
        
        return maximum
    
    def __best_hypothesis(self):
        minimum = 1
        best_hypothesis = None

        for hypothesis in self.__hypothesis:
            value = self.__fitness(hypothesis)
            if value < minimum:
                minimum = value
                best_hypothesis = hypothesis
        
        return best_hypothesis

    def __step(self):
        probabilities = self.__probabilities()
        ps = self.__select_from_current(probabilities)

        ps.extend(self.__select_pairs(probabilities))
        ps.extend(self.__sample_hypothesis(probabilities, self.__p - len(ps)))

        self.__hypothesis = self.__mutate_hypothesis(ps)
    
    def learn(self):
        contor = 1
        while self.__metric() > self.__threshold:
            self.__step()
            
            if contor % 100 == 0:
                print(str(contor) + " " + str(self.__metric())) 
            contor += 1
        
        return self.__best_hypothesis()

In [363]:
train_set = [[[1, 0, 0, 1, 0, 0, 1, 0, 0, 1], 0],
             [[1, 0, 0, 1, 0, 0, 1, 0, 1, 0], 0],
             [[0, 1, 0, 1, 0, 0, 1, 0, 0, 1], 1],
             [[0, 0, 1, 0, 1, 0, 1, 0, 0, 1], 1],
             [[0, 0, 1, 0, 0, 1, 0, 1, 0, 1], 1],
             [[0, 0, 1, 0, 0, 1, 0, 1, 1, 0], 0],
             [[0, 1, 0, 0, 0, 1, 0, 1, 1, 0], 1],
             [[1, 0, 0, 0, 1, 0, 1, 0, 0, 1], 0],
             [[1, 0, 0, 0, 0, 1, 0, 1, 0, 1], 1],
             [[0, 0, 1, 0, 1, 0, 0, 1, 0, 1], 1],
             [[1, 0, 0, 0, 1, 0, 0, 1, 1, 0], 1],
             [[0, 1, 0, 0, 1, 0, 1, 0, 1, 0], 1],
             [[0, 1, 0, 1, 0, 0, 0, 1, 0, 1], 1],
             [[0, 0, 1, 0, 1, 0, 1, 0, 1, 0], 0]]

train_instances = [element[0] for element in train_set]
train_targets = [element[1] for element in train_set]

In [364]:
fitness = Fitness(instances = train_instances, targets = train_targets)
ga = GA(length = REPRESENTATION_LENGTH, fitness = fitness)

hypothesis = ga.learn()

100 0.6428571428571429
200 0.6428571428571429
300 0.35714285714285715
400 0.35714285714285715
500 0.42857142857142855
600 0.35714285714285715
700 0.35714285714285715
800 0.35714285714285715
900 0.35714285714285715
1000 0.35714285714285715
1100 0.35714285714285715
1200 0.6428571428571429
1300 0.5


In [365]:
hypothesis.classify(instance = [1, 0, 0, 1, 0, 0, 0, 1, 1, 0])

1